# Manually rebuilding the stiffness matrix

Hermite dofs are set to zero by `bc.apply()`: we manually find those and reset the relevant rows of the mass matrix.

In the process of writing the following two functions (`find_hermite_boundary_dofs()` and `apply_dirichlet_partially()`) it was useful to look into the code for `dolfin/fem/DirichletBC.cpp, compute_bc_pointwise()`. Also there is a [bug](https://bugs.launchpad.net/dolfin/+bug/1063868) in the automatically generated documentation which affects [`GenericMatrix.set()`](https://fenicsproject.org/olddocs/dolfin/2016.2.0/python/programmers-reference/cpp/la/GenericMatrix.html?highlight=genericmatrix#dolfin.cpp.la.GenericMatrix.set) below. Only one function with three parameters is actually exposed by SWIG and the second and last are actually the indices of the rows and columns to modify.

Note that we set the linear algebra backend to Eigen because the default PETSc uses sparse matrices whose sparsity patterns are initialised / fixed by `assemble()`, so we cannot freely change entries unless we directly talk to PETSc.

This can be done with `petsc4py` (which is available in the docker container), see [this demo](https://bitbucket.org/cwilson/dolfin/src/883fcc4a12d53c7dae1bb9d5fc7e7302051b0bae/demo/undocumented/petsc4py/python/demo_petsc4py.py?at=master&fileviewer=file-view-default), but I'm too lazy now.

**Note:** Another approach could be to "fix" `DirichletBC`: It seems like I need to define a special dofmap for Hermite elements which in `dofmap.cell_dofs()` returns only the Lagrange dofs (?). Won't this break lots of other things? Instead I could try to construct a copy of the function space with the right dofmap for the BCs and pass it to DirichletBC's constructor.

In [ ]:
from dolfin import *
import FIAT
import numpy as np
import matplotlib.pyplot as pl

def find_hermite_boundary_dofs(bc):
    """ Returns the Hermite dofs at the boundary as defined in the argument.
        bc: Boundary condition.
    """
    V = bc.function_space()
    tdim = V.element().topological_dimension()
    e = FIAT.CubicHermite(FIAT.reference_element.default_simplex(tdim), 3)
    
    # This mask filters out the Hermite dofs from the list of dofs of a cell:
    mask = np.array(map(lambda f: isinstance(f, FIAT.functional.PointDerivative),
                        e.dual_basis()))

    # Compute the set of hermite dofs
    hermite_dofs = set()
    for i in range(V.mesh().num_cells()):
        hermite_dofs = hermite_dofs.union(set(V.dofmap().cell_dofs(i)[mask]))
        
    # This is an easy way of retrieving the ids of dofs at the boundary:
    boundary_dofs = bc.get_boundary_values().keys()
    
    return filter(lambda x: x in boundary_dofs, hermite_dofs)


def apply_dirichlet_hermite(A, b, bc):
    """ Applies DirichletBC to A, b in place but fixes the rows
    corresponding to Hermite dofs.
    Arguments:
        A: assembled mass matrix, before applying bc
        b: assembled right hand side, before applying bc
        bc: DirichletBC
    Returns:
        Nothing. Arguments are modified in place.
    """
    global parameters
    assert parameters['linear_algebra_backend'] == 'Eigen',\
           "Need dense matrices for now."
    assert isinstance(bc, DirichletBC),\
           "We only know how to manage Dirichlet BCs"
    
    hermite_dofs = find_hermite_boundary_dofs(bc)
    
    A0 = A.copy()
    b0 = b.copy()
    
    # Applying BCs messes things up for Hermite dofs
    bc.apply(A, b)
    
    # So we fix them now:
    
    b[hermite_dofs] = b0[hermite_dofs]
    
    block = A0.array()[hermite_dofs]
    rows = np.array(hermite_dofs, dtype=np.intc)
    cols = np.arange(A.size(1), dtype=np.intc) # FIXME A.size(1) is columns?
    A.set(block, rows, cols)
    A.apply("insert")

    
def apply_neumann_hermite(A, b, bc):
    """
    Arguments:
        A: assembled mass matrix, before applying bc
        b: assembled right hand side, before applying bc
        bc: NeumannBC (alias for DirichletBC) encoding the value of
            the normal derivative
    """
    if bc is None:
        return
    global parameters
    assert parameters['linear_algebra_backend'] == 'Eigen',\
           "Need dense matrices for now."
    assert isinstance(bc, DirichletBC),\
           "We only know how to manage Dirichlet BCs"
    assert bc.function_space().element().geometric_dimension() == 1,\
           "FIXME: I can only manage 1 dimensional normal derivatives."

    hermite_dofs = find_hermite_boundary_dofs(bc)
    
    vals = bc.get_boundary_values()  # This is a dict
    for dof in hermite_dofs:
        b[dof] = vals[dof]

    ncols = A.size(1) # FIXME A.size(1) is columns?
    nrows = len(hermite_dofs)
    block = np.zeros((nrows, ncols))
    block[range(nrows),[hermite_dofs]] = 1.
    
    rows = np.array(hermite_dofs, dtype=np.intc)
    cols = np.arange(ncols, dtype=np.intc)

    A.set(block, rows, cols)
    A.apply("insert")


def plot_hermite_dofs(bc):
    """ Plots the mesh and marks the Hermite dofs fulfilling 
    the boundary condition.
    Arguments:
        bc: DirichletBC.
    """
    V = bc.function_space()
    tdim = V.mesh().topology().dim()
    dofs = zip(V.dofmap().dofs(), V.tabulate_dof_coordinates().reshape((-1, tdim)))

    hdofs = find_hermite_boundary_dofs(bc)
    hdofs_coordinates = np.array([dof[1] for dof in filter(lambda p: p[0] in hdofs, dofs)])
    plot(V.mesh())
    if tdim > 1:
        pl.scatter(hdofs_coordinates[:,0], hdofs_coordinates[:,1],
                   s=15, c='red', linewidths=0, zorder=10)
    else:
        pl.ylim((-0.1,0.1))
        pl.scatter(hdofs_coordinates[:,0], np.zeros_like(hdofs_coordinates[:,0]),
                   c='red', linewidths=0, zorder=10)